In [1]:
#basic packages
import numpy as np 
import pandas as pd 
import os
import gc

#natural language processing in Russian
#import nltk
#nltk.download()

#term frequency
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords 

#other
from sklearn import preprocessing

In [3]:
#import raw data
training = pd.read_csv('train.csv', parse_dates = ["activation_date"])
testing = pd.read_csv('test.csv', parse_dates = ["activation_date"])
traindex = training.index
testdex = testing.index

The following data preprocessing was inspired by a kernel from Bojan Tunguz:

In [4]:
#feature engineering:price (training)
training["price"] = np.log(training["price"]+0.001)
training["price"].fillna(-999,inplace=True)
training["image_top_1"].fillna(-999,inplace=True)

training["Weekday"] = training['activation_date'].dt.weekday
training["Weekd of Year"] = training['activation_date'].dt.week
training["Day of Month"] = training['activation_date'].dt.day

#feature engineerin:price (testing)
testing["price"] = np.log(testing["price"]+0.001)
testing["price"].fillna(-999,inplace=True)
testing["image_top_1"].fillna(-999,inplace=True)

testing["Weekday"] = testing['activation_date'].dt.weekday
testing["Weekd of Year"] = testing['activation_date'].dt.week
testing["Day of Month"] = testing['activation_date'].dt.day

In [5]:
#create validation index, remove variables we won't use, and encode remaining variables
#training
training_index = training.loc[training.activation_date<=pd.to_datetime('2017-04-07')].index
train_val_index = training.loc[training.activation_date>=pd.to_datetime('2017-04-08')].index
training.drop(["activation_date","image"],axis=1,inplace=True)

categorical = ["user_id","region","city","parent_category_name","category_name","user_type","image_top_1"]
lbl = preprocessing.LabelEncoder()
for col in categorical:
    training[col] = lbl.fit_transform(training[col].astype(str))

#testing
testing_index = testing.loc[testing.activation_date<=pd.to_datetime('2017-04-07')].index
test_val_index = testing.loc[testing.activation_date>=pd.to_datetime('2017-04-08')].index
testing.drop(["activation_date","image"],axis=1,inplace=True)

#categorical = ["user_id","region","city","parent_category_name","category_name","user_type","image_top_1"] already defined above
lbl = preprocessing.LabelEncoder()
for col in categorical:
    testing[col] = lbl.fit_transform(testing[col].astype(str))

In [6]:
#feature engineering: text (training)
training['text_feat'] = training.apply(lambda row: ' '.join([
    str(row['param_1']), 
    str(row['param_2']), 
    str(row['param_3'])]),axis=1) # Group Param Features
training.drop(["param_1","param_2","param_3"],axis=1,inplace=True)

#feature engineering: text (testing)
testing['text_feat'] = testing.apply(lambda row: ' '.join([
    str(row['param_1']), 
    str(row['param_2']), 
    str(row['param_3'])]),axis=1) # Group Param Features
testing.drop(["param_1","param_2","param_3"],axis=1,inplace=True)

In [7]:
#meta text features (training)
textfeats = ["description","text_feat", "title"]
for cols in textfeats:
    training[cols] = training[cols].astype(str) 
    training[cols] = training[cols].astype(str).fillna('nicapotato') # FILL NA
    training[cols] = training[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    training[cols + '_num_chars'] = training[cols].apply(len) # Count number of Characters
    training[cols + '_num_words'] = training[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    training[cols + '_num_unique_words'] = training[cols].apply(lambda comment: len(set(w for w in comment.split())))
    training[cols + '_words_vs_unique'] = training[cols+'_num_unique_words'] / training[cols+'_num_words'] * 100 # Count Unique Words
    
#meta text features (testing)
#textfeats = ["description","text_feat", "title"], already defined above
for cols in textfeats:
    testing[cols] = testing[cols].astype(str) 
    testing[cols] = testing[cols].astype(str).fillna('nicapotato') # FILL NA
    testing[cols] = testing[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    testing[cols + '_num_chars'] = testing[cols].apply(len) # Count number of Characters
    testing[cols + '_num_words'] = testing[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    testing[cols + '_num_unique_words'] = testing[cols].apply(lambda comment: len(set(w for w in comment.split())))
    testing[cols + '_words_vs_unique'] = testing[cols+'_num_unique_words'] / testing[cols+'_num_words'] * 100 # Count Unique Words

In [8]:
#set stopwords
russian_stop = set(stopwords.words('russian'))

#term frequency parameters
tfidf_para = {
    "stop_words": russian_stop,
    "analyzer": 'word',
    "token_pattern": r'\w{1,}',
    "sublinear_tf": True,
    "dtype": np.float32,
    "norm": 'l2',
    #"min_df":5,
    #"max_df":.9,
    "smooth_idf":False
}

#define function to create term frequency variables
def get_col(col_name): return lambda x: x[col_name]

#create vectorized NLP variables
vectorizer = FeatureUnion([
        ('description',TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=16000,
            **tfidf_para,
            preprocessor=get_col('description'))),
        ('text_feat',CountVectorizer(
            ngram_range=(1, 2),
            #max_features=7000,
            preprocessor=get_col('text_feat'))),
        ('title',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            #max_features=7000,
            preprocessor=get_col('title')))
    ])

In [9]:
training.head()

,item_id,user_id,region,city,parent_category_name,category_name,title,description,price,item_seq_number,...,description_num_unique_words,description_words_vs_unique,text_feat_num_chars,text_feat_num_words,text_feat_num_unique_words,text_feat_words_vs_unique,title_num_chars,title_num_words,title_num_unique_words,title_words_vs_unique
0,b912c3c6a6ad,675853,19,460,4,42,кокоби(кокон для сна),"кокон для сна малыша,пользовались меньше месяц...",5.991467,2,...,7,100.0,33,4,3,75.000000,21,3,3,100.0
1,2dac0150717d,173962,17,1300,2,22,стойка для одежды,"стойка для одежды, под вешалки. с бутика.",8.006368,19,...,7,100.0,14,3,2,66.666667,17,3,3,100.0
2,ba83aefab5dc,440069,16,1276,0,2,philips bluray,"в хорошем состоянии, домашний кинотеатр с blu ...",8.294050,9,...,17,100.0,35,7,6,85.714286,14,2,2,100.0
3,02996f1dd2ea,576929,21,940,4,42,автокресло,продам кресло от0-25кг,7.696213,286,...,3,100.0,28,4,3,75.000000,10,1,1,100.0
4,7c90be56d2ab,721853,4,317,6,0,"ваз 2110, 2003",все вопросы по телефону.,10.596635,3,...,4,100.0,26,5,5,100.000000,14,3,3,100.0


### Add counts of deal or no deal words

In [10]:
#create lists of top words
dealY = ('состоянии', 'продам', 'хорошем', 'состояние', 'отличном', 'торг', 'очень', 'продаю', 'размер', 'цена', 'б/у', 'работает', 'новый', 'хорошее', 'отличное', 'телефону', 'года', 'фото', 'новые', 'отдам', 'новая', 'комплекте', 'машина', 'продается', 'вопросы', 'цвет', 'ремонт', 'лет', 'квартира', 'идеальном', 'квартиру', 'комплект', 'работы', 'авто', 'руб', 'автомобиль', 'рабочем', 'коробка', 'телефон', 'срок', 'сдам', 'возможен', 'звоните', 'полностью', 'обмен', 'дом', 'коляска', 'велосипед', '"продам', 'двигатель', 'длительный', 'срочно', 'подарок', 'имеется', 'самовывоз', 'полный', 'холодильник', 'резина', 'музыка', 'время', 'сигнализация', 'диван', 'длина', 'дома', 'салон', 'всё', 'связи', 'документы', 'любой', 'отлично', 'рублей', 'высота', 'продаётся', 'чехол', 'колеса', 'мальчика', 'большой', 'детский', 'требует', 'рядом', 'стоит', 'девочки', 'пробег', 'кожа', 'год', 'фирмы', 'идеальное', 'коляску', 'мало', 'мебель', 'также', 'ходу', 'цвета', 'уместен', 'услуги', 'птс', 'торга', 'рост', 'оригинал', 'стол')
dealN = ('состоянии', 'продам', 'размер', 'состояние', 'хорошем', 'очень', 'отличном', 'торг', 'продаю', 'новые', 'цена', 'новый', 'отличное', 'новая', 'хорошее', 'платье', 'дом', 'б/у', 'квартира', 'цвет', 'продается', 'руб', 'фото', 'рост', 'года', 'идеальном', 'квартиру', 'лет', 'рядом', 'кожа', 'телефону', 'длина', 'участок', 'цвета', 'мальчика', 'возможен', 'ремонт', 'куртка', 'окна', 'дома', 'вопросы', 'девочки', 'работает', 'новое', 'звоните', 'комплект', 'рублей', 'натуральная', 'костюм', 'доме', 'носили', 'пару', 'комплекте', 'подарок', 'отдам', 'обмен', 'имеется', 'комбинезон', 'туфли', 'телефон', 'девочку', 'идеальное', 'районе', 'пальто', 'кухня', 'комнаты', 'уместен', 'наличии', 'внутри', 'документы', 'район', 'стельке', 'продаётся', 'большой', 'фирмы', 'школа', 'две', 'срочно', 'полностью', 'мало', 'джинсы', 'вода', 'раза', 'качество', 'высота', 'города', 'сапоги', 'ботинки', 'машина', 'гараж', 'покупала', 'удобные', 'магазины', 'покупали', 'работы', 'также', 'время', 'газ', 'пишите', 'детский',)

In [11]:
#eliminiate words that are in both lists
dealY_unique = [elem for elem in dealY if elem not in dealN]
dealN_unique = [elem for elem in dealN if elem not in dealY]

In [12]:
#create and apply word count functions to data 
wcf_h = lambda d: [sum(d.count(x) for x in dealY_unique)]
training['hi_dp_wordcount'] = training['description'].apply(wcf_h)
testing['hi_dp_wordcount'] = testing['description'].apply(wcf_h)

wcf_l = lambda d: [sum(d.count(x) for x in dealN_unique)]
training['lo_dp_wordcount'] = training['description'].apply(wcf_l)
testing['lo_dp_wordcount'] = testing['description'].apply(wcf_l)

In [26]:
#fix format for series
training['hi_dp_wordcount'] = training['hi_dp_wordcount'].apply(pd.Series).stack().reset_index(drop=True)
training['lo_dp_wordcount'] = training['lo_dp_wordcount'].apply(pd.Series).stack().reset_index(drop=True)
testing['hi_dp_wordcount'] = testing['hi_dp_wordcount'].apply(pd.Series).stack().reset_index(drop=True)
testing['lo_dp_wordcount'] = testing['lo_dp_wordcount'].apply(pd.Series).stack().reset_index(drop=True)

In [27]:
training.head()

,item_id,user_id,region,city,parent_category_name,category_name,title,description,price,item_seq_number,...,text_feat_num_chars,text_feat_num_words,text_feat_num_unique_words,text_feat_words_vs_unique,title_num_chars,title_num_words,title_num_unique_words,title_words_vs_unique,hi_dp_wordcount,lo_dp_wordcount
0,b912c3c6a6ad,675853,19,460,4,42,кокоби(кокон для сна),"кокон для сна малыша,пользовались меньше месяц...",5.991467,2,...,33,4,3,75.000000,21,3,3,100.0,0,0
1,2dac0150717d,173962,17,1300,2,22,стойка для одежды,"стойка для одежды, под вешалки. с бутика.",8.006368,19,...,14,3,2,66.666667,17,3,3,100.0,0,0
2,ba83aefab5dc,440069,16,1276,0,2,philips bluray,"в хорошем состоянии, домашний кинотеатр с blu ...",8.294050,9,...,35,7,6,85.714286,14,2,2,100.0,0,0
3,02996f1dd2ea,576929,21,940,4,42,автокресло,продам кресло от0-25кг,7.696213,286,...,28,4,3,75.000000,10,1,1,100.0,0,0
4,7c90be56d2ab,721853,4,317,6,0,"ваз 2110, 2003",все вопросы по телефону.,10.596635,3,...,26,5,5,100.000000,14,3,3,100.0,0,0


### Add new features from Minxuan (image features, city population, sentiment analysis):

In [28]:
#load datasets with new features (prepared in other code)
NF_train = pd.read_csv('newtrainfeatures.csv')
NF_test = pd.read_csv('newtestfeatures.csv')

In [29]:
NF_train.head()

,item_id,perform_white_analysis,perform_black_analysis,image_size,average_pixel_width,get_blurrness_score,average_red,average_green,average_blue,descsentiment,titlesentiment,reg_dense,rural,reg_Time_zone,reg_Population,reg_Urban,city_population
0,b912c3c6a6ad,1.965,0.000,838,2.338222,398.109961,0.36,0.32,0.61,0.0,0.0,22.06,16.1,YEKT,4329341,83.9,1455904.0
1,2dac0150717d,0.000,6.335,840,3.240162,1014.604608,0.30,0.37,0.42,0.0,0.0,59.99,19.8,SAMT,3212676,80.2,1169719.0
2,ba83aefab5dc,72.020,0.000,752,2.643849,493.921065,0.70,0.70,0.70,0.0,0.0,42.44,32.8,MSK,4254600,67.2,1125299.0
3,02996f1dd2ea,94.325,0.000,720,1.524691,377.105718,0.85,0.85,0.85,0.0,0.0,55.68,24.6,MSK,3838374,75.4,529797.0
4,7c90be56d2ab,1.320,7.715,1000,2.475260,557.351080,0.45,0.51,0.49,-1.0,0.0,22.92,24.0,MSK,2569126,76.0,1015586.0


In [30]:
#merge datasets on item_id to include all new features
finaltrain = pd.merge(training, NF_train, on='item_id')
finaltest = pd.merge(testing, NF_test, on='item_id')

In [31]:
finaltest.head()

,item_id,user_id,region,city,parent_category_name,category_name,title,description,price,item_seq_number,...,average_green,average_blue,descsentiment,titlesentiment,reg_dense,rural,reg_Time_zone,reg_Population,reg_Urban,city_population
0,6544e41a8817,262938,4,306,4,10,отдам бесплатно,на ангарском,-999.000000,66,...,0.42,0.42,0.0,1.0,22.92,24.0,MSK,2569126,76.0,1015586.0
1,65b9484d670f,55145,19,933,8,5,продам велосипед,"продам велосипед kama f200,в нормальном состо...",8.006368,4,...,-999.00,-999.00,-1.0,0.0,22.06,16.1,YEKT,4329341,83.9,20217.0
2,8bab230b2ecd,13649,12,147,0,2,bbk,продам новый телевизор bbk 32 диагональ смарт...,9.615806,15,...,0.30,0.28,1.0,0.0,14.96,22.7,KRAT,2746822,77.3,103290.0
3,8e348601fefc,113959,18,1238,2,4,вытяжка jetair 60,"продам новую вытяжку в упаковке,с документами....",8.411833,70,...,-999.00,-999.00,0.0,0.0,25.17,25.5,SAMT,2496600,74.5,845300.0
4,8bd2fe400b89,42880,14,236,4,42,коляска зима-лето,продам отличную коляску. б/у 1 год. все вопрос...,8.496991,15,...,0.27,0.36,-1.0,0.0,16.40,40.3,YEKT,2033072,59.7,86316.0


In [32]:
#export to CSV for use in analysis
finaltrain.to_csv('finaltrain.csv', index=False, header=True, encoding='utf-8')
finaltest.to_csv('finaltest.csv', index=False, header=True, encoding='utf-8')